****************************************************************

# Data exploration

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Quick-note on project directory

The main root dir `~/3dcorrection` is structured as follow:
* `data/` contains raw and preprocessed data. 
    * `raw/` is actually a symbolic link to the same repo for all candidates, DO NOT TOUCH IT!
    * `processed/` will be created when data is preprocessed and will contain all transformed data
* 

In [2]:
import os

root_path = os.path.join('/', 'home', 'jupyter', 'bootcamps')

data_path = os.path.join(root_path, 'data')
cache_path = os.path.join(data_path, 'cache')
raw_data_path = os.path.join(data_path, 'raw')
processed_data_path = os.path.join(data_path, 'processed')

### The 3D Correction Use-Case

The European Centre for Medium-range Weather Forecasts (ECMWF) has developed a series of model giving the current best accurate parametrization scheme available—among those, SPARTACUS delivers **radiation** prediction over the globe. Because it is demanding in computations, a simpler, degraded model called TRIPLECLOUD is developed to satisfy the production environment constraints. 

Like most climate models, to leverage hardware acceleration, the choice is made to split the globe in blocks—this has the immediate consequence of losing the spatial correlation for a gain in parallelization. 

The unit block is a column that express values throughout the vertical dimension over a set of levels. Each level is

Now let's load the raw data we'll be using throughout this hands-on. Take a look at the [source notebook](https://git.ecmwf.int/projects/MLFET/repos/maelstrom-radiation/browse/climetlab_maelstrom_radiation/radiation.py) for a more info on the variables.

In [3]:
7 // 3

2

In [4]:
7 % 3

1

In [5]:
import numpy as np
import os.path as osp

d = np.arange(15, dtype='float32')
d = d.reshape((3, -1))

In [6]:
d

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.]], dtype=float32)

In [7]:
path = osp.join(processed_data_path, 'test.data.npy')
np.save(path, d)

In [8]:
np.memmap(path, mode='r', dtype='float32', shape=(3, 5))

memmap([[2.2366853e+08, 1.2387478e-40, 3.4833497e-15, 4.4898648e+21,
         1.5767864e-19],
        [2.1442303e-07, 2.2228396e-15, 7.6830766e+31, 1.7177136e+19,
         6.7425655e+22],
        [1.5767864e-19, 1.8727951e+31, 2.2228527e-15, 2.7904159e+29,
         1.5767847e-19]], dtype=float32)

In [9]:
np.load(path)

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.]], dtype=float32)

In [10]:
import data
import models

In [11]:
datamodule = data.LitThreeDCorrectionDataModule(250, 256, 0)

In [12]:
model = models.LitGAT(
    in_channels=20,
    hidden_channels=100,
    out_channels=4,
    num_layers=2,
    dropout=.5,
    heads=4,
    jk='last',
    lr=1e-3,)

In [13]:
import pytorch_lightning as pl

trainer = pl.Trainer()
trainer.fit(model=model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type | Params
------------------------------
0 | net  | GAT  | 2.7 K 
------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/bootcamps/0_data_preprocessing/../data/processed/feats-250/7565.npy'